<a href="https://colab.research.google.com/github/KamonohashiPerry/MachineLearning/blob/master/Causal_Inference/Python_Causal_Inference_Chap7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ベイジアンネットワーク
+ DAGを前提としている
+ グラフ表現により変数間の関係性を示すための手法
+ 構造方程式は使わないで、条件付き確率表を用いる。

### 関連用語
+ スケルトン
 + ノードとノードの繋がりだけを表す（やじるしなし）
+ PDAG
 + 一部は有向で一部が無向のグラフ

### ネットワークの当てはまりの良さ
+ AIC
+ BIC
+ BDeu
+ MDL符号
+ BDe
+ BDeu
+ K2

In [2]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

from numpy.random import *
import pandas as pd

データの作成

In [4]:
# データ数
num_data = 10

# x1:0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2:0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df = pd.DataFrame({'x1':x1,
                    'x2':x2})
df.head()

,x1,x2
0,0,1
1,1,1
2,0,1
3,0,0
4,1,1


In [7]:
# 変数x3:0か1の値をnum_data個生成する
# (x1, x2) = (0, 0)のとき、0の確率は0.2
# (x1, x2) = (1, 0)のとき、0の確率は0.3
# (x1, x2) = (0, 1)のとき、0の確率は0.4
# (x1, x2) = (1, 1)のとき、0の確率は0.1

x3 = []
for i in range(num_data):
  if x1[i] == 0 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.2, 0.8])
    x3.append(x3_value[0])
  elif x1[i] == 0 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.3, 0.7])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.4, 0.6])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.1, 0.9])
    x3.append(x3_value[0])

df['x3'] = x3

df

,x1,x2,x3
0,0,1,1
1,1,1,0
2,0,1,1
3,0,0,1
4,1,1,1
5,1,1,1
6,0,0,1
7,1,1,1
8,0,1,0
9,0,1,1


In [10]:
 !pip install pgmpy==0.1.9

     |████████████████████████████████| 337kB 3.4MB/s 


In [11]:
# 正解のDAGを与える
from pgmpy.models import BayesianModel

model = BayesianModel([('x1', 'x3'), ('x2', 'x3')])

In [12]:
# 各データパターンの個数を表示する
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('x1'))
print("\n", pe.state_counts('x2'))
print("\n", pe.state_counts('x3'))


    x1
0   6
1   4

    x2
0   2
1   8

 x1    0         1     
x2    0    1    0    1
x3                    
0   0.0  1.0  0.0  1.0
1   2.0  3.0  0.0  3.0


In [13]:
# CPT（条件付き確率表）を推定する
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd('x1', prior_type='dirichlet', pseudo_counts=[[0], [0]])
cpd_x2 = estimator.estimate_cpd('x2', prior_type='dirichlet', pseudo_counts=[[0], [0]])
cpd_x3 = estimator.estimate_cpd('x3', prior_type='dirichlet', pseudo_counts=[[0,0,0,0], [0,0,0,0]])

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)

+-------+-----+
| x1(0) | 0.6 |
+-------+-----+
| x1(1) | 0.4 |
+-------+-----+
+-------+-----+
| x2(0) | 0.2 |
+-------+-----+
| x2(1) | 0.8 |
+-------+-----+
+-------+-------+-------+-------+-------+
| x1    | x1(0) | x1(0) | x1(1) | x1(1) |
+-------+-------+-------+-------+-------+
| x2    | x2(0) | x2(1) | x2(0) | x2(1) |
+-------+-------+-------+-------+-------+
| x3(0) | 0.0   | 0.25  | nan   | 0.25  |
+-------+-------+-------+-------+-------+
| x3(1) | 1.0   | 0.75  | nan   | 0.75  |
+-------+-------+-------+-------+-------+


/usr/local/lib/python3.6/dist-packages/pgmpy/factors/discrete/CPD.py:298: RuntimeWarning: invalid value encountered in true_divide
  tabular_cpd.values = (cpd / cpd.sum(axis=0)).reshape(tabular_cpd.cardinality)


In [14]:
# BICを求める
from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))

-23.140577341407056


In [15]:
# 正解ではないDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x2', 'x1'), ('x2', 'x3')])
bic = BicScore(df)
print(bic.score(model))

-20.804345569297023
